In [62]:
# All Features Dimension = 5183

# ECFP(1024) + Pubchem FP(881) + Descriptors(11)=1916

# Descriptors calculated by mordred (1826)

# Interaction Fingerprint(1440) + Docking Score (1)=1441

5183-1916-1826-1441

0

ML

In [63]:
from featurization import compute_fps, compute_physchem, compute_fingerprint_features, compute_descriptor_features
import numpy as np
from typing import List
import pandas as pd
import pickle
%reload_ext autoreload
%autoreload 2
from new_unit import Data, get_config, cross_validate, f1_acc
from models import RF, SVM, GBM,  GCN, MPNN, CNN, Transformer
from rdkit.Chem import Descriptors as Des
from rdkit import Chem

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,precision_recall_curve, auc
from math import sqrt
from unit import compute_metrics
np.random.seed(42)

In [64]:
Moldel_Type = "ML"
Model = "SVM"
FingerPrint = "2FP"
Des_IF = "11Des_1441IF"

file_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}.csv"
model_name = f"./trained_models/{Moldel_Type}/{Model}_{FingerPrint}_{Des_IF}.pkl"

# hyperparameters = get_config(f"./configures/optimized/train_split/{Model}_ECFP.yml")
# if "epochs" in hyperparameters:
#     hyperparameters.pop("epochs")
# f = GBM(**hyperparameters)

with open(model_name, 'rb') as handle:
    f = pickle.load(handle)

In [65]:
with open(file_name,"w") as p:
    p.write("auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC\n")

In [66]:
df = pd.read_csv("./data/IF_Data/Train_Val_IF.csv")
df['r_i_docking_score'] = df['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df["MW"]=df["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))


df_train = df[df["split"]=="train"].reset_index()
df_val = df[df["split"]=="test"].reset_index()

train_mask = (df_train['MW'] >= 300) & (df_train['MW'] <= 500)
val_mask = (df_val['MW'] >= 300) & (df_val['MW'] <= 500)

train_Ro5_idx = df_train[train_mask].index
val_Ro5_idx = df_val[val_mask].index


In [67]:
df_Test_1 = pd.read_csv("./data/IF_Data/Test_1_IF.csv")
df_Test_1['r_i_docking_score'] = df_Test_1['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df_Test_1["MW"]=df_Test_1["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))

# df_Test_1 = df_Test_1.reset_index()

Test_1_mask = (df_Test_1['MW'] >= 300) & (df_Test_1['MW'] <= 500)
Test_1_Ro5_idx = df_Test_1[Test_1_mask].index

In [68]:
Test_1_Ro5_idx.shape

(177,)

In [69]:
df_Test_2 = pd.read_csv("./data/IF_Data/Test_2_IF.csv")
df_Test_2['r_i_docking_score'] = df_Test_2['r_i_docking_score'].apply(lambda x: 0 if x > 0 else x)
df_Test_2["MW"]=df_Test_2["smiles"].apply(lambda x : Des.ExactMolWt(Chem.MolFromSmiles(x)))

# df_Test_2 = df_Test_2.reset_index()

Test_2_mask = (df_Test_2['MW'] >= 300) & (df_Test_2['MW'] <= 500)
Test_2_Ro5_idx = df_Test_2[Test_2_mask].index

In [70]:
Test_2_Ro5_idx.shape

(379,)

In [71]:
train_all_rep = "./data/IF_Data/Train_All_Features.pkl"
with open(train_all_rep, 'rb') as handle:
    train_all_rep = pickle.load(handle)[train_Ro5_idx]

val_all_rep = "./data/IF_Data/Val_All_Features.pkl"
with open(val_all_rep, 'rb') as handle:
    val_all_rep = pickle.load(handle)[val_Ro5_idx]

Test_1_all_rep = "./data/IF_Data/Test_1_All_Features.pkl"
with open(Test_1_all_rep, 'rb') as handle:
    Test_1_all_rep = pickle.load(handle)[Test_1_Ro5_idx]

Test_2_all_rep = "./data/IF_Data/Test_2_All_Features.pkl"
with open(Test_2_all_rep, 'rb') as handle:
    Test_2_all_rep = pickle.load(handle)[Test_2_Ro5_idx]

In [72]:
Test_1_all_rep.shape

(177, 5183)

In [73]:
Test_2_all_rep.shape

(379, 5183)

In [74]:
# Train
# f.train(train_all_rep[:,:1916], np.array(df_train[train_mask]["y"]))
# with open(model_name, 'wb') as handle:
#     pickle.dump(f, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
### Validation

# y_hat = f.predict(val_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(val_all_rep[:,:1916])[:, 1]

y_hat = f.predict(np.hstack((val_all_rep[:, :1916], val_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((val_all_rep[:, :1916], val_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_val[val_mask]["y"], y_hat_proba, file_name)

f1, acc = f1_acc(np.array(df_val[val_mask]["y"]), y_hat)
fpr, tpr, thresholds = roc_curve(np.array(df_val[val_mask]["y"]), y_hat_proba)
roc_auc = auc(fpr, tpr)
print(f1)

Binary classification performace metrics:
[[574  96]
 [130 317]]
TP, FN, TN, FP
317, 130, 574, 96
0.7292418772563177


In [76]:
### Test 1

# y_hat = f.predict(Test_1_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(Test_1_all_rep[:,:1916])[:, 1]
y_hat = f.predict(np.hstack((Test_1_all_rep[:, :1916], Test_1_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((Test_1_all_rep[:, :1916], Test_1_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_Test_1[Test_1_mask]["y"], y_hat_proba, file_name)
f1, acc = f1_acc(np.array(df_Test_1[Test_1_mask]["y"]), y_hat)
fpr, tpr, thresholds = roc_curve(np.array(df_Test_1[Test_1_mask]["y"]), y_hat_proba)
roc_auc = auc(fpr, tpr)
print(f1)

Binary classification performace metrics:
[[116  13]
 [ 28  20]]
TP, FN, TN, FP
20, 28, 116, 13
0.5


In [77]:
### Test 2

# y_hat = f.predict(Test_2_all_rep[:,:1916])
# y_hat_proba = f.predict_prob(Test_2_all_rep[:,:1916])[:, 1]
y_hat = f.predict(np.hstack((Test_2_all_rep[:, :1916], Test_2_all_rep[:, 3742:])))
y_hat_proba = f.predict_prob(np.hstack((Test_2_all_rep[:, :1916], Test_2_all_rep[:, 3742:])))[:, 1]

auc_score, pr_auc, AC, f1, SN, PR, SP, CCR, MCC = compute_metrics(df_Test_2[Test_2_mask]["y"], y_hat_proba, file_name)
f1, acc = f1_acc(np.array(df_Test_2[Test_2_mask]["y"]), y_hat)
fpr, tpr, thresholds = roc_curve(np.array(df_Test_2[Test_2_mask]["y"]), y_hat_proba)
roc_auc = auc(fpr, tpr)
print(f1)

Binary classification performace metrics:
[[247  28]
 [ 51  53]]
TP, FN, TN, FP
53, 51, 247, 28
0.5810055865921788
